Import some library code


*   `tensorflow` is the machine learning library we're using
*   `os` helps us interact with files and folders
*   `matplotlib` is for displaying charts and images
*   `numpy` helps us work with data to prepare it for `tensorflow` and review it afterwards


In [202]:
import tensorflow as tf
import os
import matplotlib.pyplot as plt
import numpy as np
import validators
import shutil

# print(tf.__version__)
# physical_devices = tf.config.list_physical_devices('GPU')
# tf.print(physical_devices)

This code gets the data you will use to train your model: pictures of cats and dogs.


In [203]:
import tensorflow_datasets as tfds

(raw_training, raw_validation, raw_testing), metadata = tfds.load( 
    name=tfds.image_classification.CatsVsDogs.name, 
    split=['train[:80%]', 'train[80%:90%]', 'train[90%:]'], 
    with_info=True, 
    as_supervised=True)


This function takes an image and a label as inputs. The image is then converted into three sets of numbers representing the colours red, green, and blue for every pixel in the image. The combination of these colours can form any other colour. The red, green, and blue values are then converted from numbers between 0 and 255, to numbers between -1 and 1, as the model has been trained to work with values in that range. Finally, the image is resized based on the `IMAGE_SIZE` constant, to match the size the model was previously trained on. In this case, it's a 160 by 160 pixel square.


In [204]:
IMAGE_SIZE = 160

training_data = None

# Resize an image, and convert it into a form that tensorflow can read more easily 
def prep_image(image, label):
  image = tf.cast(image, tf.float32)
  image = (image/127.5) - 1
  image = tf.image.resize(image, (IMAGE_SIZE, IMAGE_SIZE))
  print("Label: " + str(label))
  return image, label

training_data = raw_training.map(prep_image)
validation_data = raw_validation.map(prep_image)
testing_data = raw_testing.map(prep_image)

validation_data.map(print)

Label: Tensor("args_1:0", shape=(), dtype=int64)
Label: Tensor("args_1:0", shape=(), dtype=int64)
Label: Tensor("args_1:0", shape=(), dtype=int64)


<MapDataset element_spec=TensorSpec(shape=<unknown>, dtype=tf.int32, name=None)>

These are versions of the functions from the previous project, so you can use them to test your model.

In [205]:
def get_image_from_url(image_uri):
  # Modify path as appropiate!
  test_image = '/Users/michaelmcdowell/.keras/datasets/test_image.jpg'
  
  # If the temporary test_image.jpg file already exists, 
  # delete it so a new one can be made.
  if os.path.exists(test_image):
    os.remove(test_image)

  if validators.url(image_uri):
    image_path = tf.keras.utils.get_file('test_image.jpg', origin=image_uri)
  else:
    image_path = shutil.copyfile(image_uri, test_image)

  return image_path

def print_predictions(predictions):
    for (prediction, number) in zip(predictions[0], range(1, len(predictions[0])+1)):
      print('{}. {} {:.2f}%'.format(number, prediction[1], prediction[2]*100))

def show_image(image):
  plt.figure()
  plt.imshow(image)

def predict_image(image_url):
  image_path = get_image_from_url(image_url)
  
  image = tf.keras.preprocessing.image.load_img(image_path, target_size=(IMAGE_SIZE, IMAGE_SIZE))

  image_arr = tf.keras.preprocessing.image.img_to_array(image)
  image_vector = np.expand_dims(image_arr, axis=0)
  
  prediction_result = history.model.predict(image_vector, batch_size=1)
  labels = metadata.features['label'].names
  print(labels[prediction_result.argmin()])

  return image

Import and test the MobileNetV2 model that you will retrain. (Issues **str not in decode fixed** _conda install 'h5py==2.10.0'_)

In [206]:
IMAGE_SHAPE = (IMAGE_SIZE, IMAGE_SIZE, 3)
original_model = tf.keras.applications.MobileNetV2(input_shape=IMAGE_SHAPE, include_top=False)
original_model.trainable = False

In the cell below, split your images into training, validation, and testing data.

In [207]:
BATCH_SIZE = 32
SHUFFLE_BUFFER_SIZE = 1000

training_batches = training_data.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
validation_batches = validation_data.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
testing_batches = testing_data.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)

Add the new layers to the model, to allow it to be retrained.

In [208]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
prediction_layer = tf.keras.layers.Dense(2)

model = tf.keras.Sequential([
  original_model,
  global_average_layer,
  prediction_layer
])

BASE_LEARNING_RATE = 0.001
model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=BASE_LEARNING_RATE),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

model.summary()

Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_160 (Funct  (None, 5, 5, 1280)       2257984   
 ional)                                                          
                                                                 
 global_average_pooling2d_22  (None, 1280)             0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dense_17 (Dense)            (None, 2)                 2562      
                                                                 
Total params: 2,260,546
Trainable params: 2,562
Non-trainable params: 2,257,984
_________________________________________________________________


Set up your training epochs and train the new layers of the model.

In [209]:
TRAINING_EPOCHS = 10
# tf.config.set_visible_devices(physical_devices[0], 'CPU')
with tf.device('device:GPU:0'):
    history = model.fit(training_batches,
                        epochs=TRAINING_EPOCHS,
                        validation_data=validation_batches)

Epoch 1/10


2022-04-01 16:52:41.508474: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


168/582 [=======>......................] - ETA: 17s - loss: 0.7594 - accuracy: 0.6845

Corrupt JPEG data: 99 extraneous bytes before marker 0xd9


202/582 [=========>....................] - ETA: 16s - loss: 0.7479 - accuracy: 0.6912

...

549/582 [===========================>..] - ETA: 1s - loss: 0.4347 - accuracy: 0.9596

Corrupt JPEG data: 228 extraneous bytes before marker 0xd9


581/582 [============================>.] - ETA: 0s - loss: 0.4344 - accuracy: 0.9600

Corrupt JPEG data: 162 extraneous bytes before marker 0xd9
Corrupt JPEG data: 252 extraneous bytes before marker 0xd9
Corrupt JPEG data: 214 extraneous bytes before marker 0xd9


582/582 [==============================] - 25s 42ms/step - loss: 0.4342 - accuracy: 0.9601 - val_loss: 0.4416 - val_accuracy: 0.9553


Use the `predict_image` function to test your model.

In [ ]:
dog1='https://i.ibb.co/Y2s0WH6/test-dog.jpg'
dog2='https://activerain-store.s3.amazonaws.com/image_store/uploads/1/5/9/3/8/ar128364712183951.jpg'
dog3='https://img.dog-learn.com/dog-breeds/bernese-mountain-dog/images/bernese-mountain-dog-r.jpg'
dog4='http://www.german-shepherd-lore.com/images/xfluffy-the-wardog.jpg.pagespeed.ic.aBrl6wfDT4.jpg'
cat1= 'https://mytabcat.com/wp-content/uploads/2015/08/Lifestyle-1-180x180.jpg'
cat2= 'https://mytabcat.com/wp-content/uploads/2015/08/Lifestyle-2-180x180.jpg'
cat3= 'https://mytabcat.com/wp-content/uploads/2015/08/Lifestyle-4-180x180.jpg'
cat4= 'https://mytabcat.com/wp-content/uploads/2015/08/Lifestyle-5-180x180.jpg'
elephant= '/Users/michaelmcdowell/Code/SimpleClassifier/elephant.jpg'

def combine(b, c):
    def a(x):
        return c(b(x))
    return a

predict_and_show = combine(predict_image, show_image)

In [ ]:
predict_and_show(dog1)

Test images from the original Cat Vs Dogs dataset.

In [ ]:
import random
def test_images(location):
    LOOP = 10 # images to test

    for n in range(1, LOOP):
        prefx = random.randrange(1, 10000)
        numStr = str(prefx)
        fName = numStr + '.jpg'
        target = location + fName
        print("Target: "+target)
        predict_image(target)

test_images('/Users/michaelmcdowell/Code/data/kagglecatsanddogs/PetImages/Cat/')